# Problem Definition

## Dataset:
Consists of all even numbers between zero and 2^n as binary representations

Example number 56 is 0111000.

This will allow us to test the performance of the generator easily.

## Generator

takes in random noise and learns to produce only even numbers.

Noise Variants:

*   Random interger in the range (0,10) for every bit
*   A single random interger in the range (0,10) 
*   A single random floating value in the range (0,1) for every bit




---




# Convert a number to binary

In [33]:
def get_binary_representation(number, max_number = 128):
  number_of_bits = int(math.log(max_number, 2))
  code = "{0:0" + str(number_of_bits) + "b}"
  string_representation = code.format(number)
  return [int(char) for char in string_representation]


get_binary_representation(1)

[0, 0, 0, 0, 0, 0, 1]

# Generate data for training

In [41]:
import math
import torch
import numpy as np
import random

# Generates #batch_size even numbers in the range 0 to max_number
def generate_training_data(max_number = 128, batch_size = 16):
  
  data = []
  labels = [1] * batch_size
  for x in range(batch_size):
    data.append(get_binary_representation(random.randrange(0, max_number-1, 2)))
	
  return torch.tensor(labels).float(), torch.tensor(data).float()

generate_training_data()

(tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 tensor([[1., 0., 1., 0., 1., 0., 0.],
         [0., 1., 1., 1., 0., 1., 0.],
         [1., 1., 0., 0., 0., 1., 0.],
         [0., 1., 1., 1., 1., 0., 0.],
         [1., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 1., 1., 0.],
         [1., 1., 0., 0., 1., 0., 0.],
         [1., 1., 0., 0., 1., 1., 0.],
         [0., 1., 0., 1., 0., 1., 0.],
         [1., 1., 1., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0., 1., 0.],
         [1., 0., 0., 1., 1., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 1., 1., 1., 0.],
         [1., 0., 0., 0., 1., 0., 0.],
         [0., 1., 0., 1., 1., 1., 0.]]))

# Generator and Discriminator

In [0]:
import torch.nn as nn

class Generator(nn.Module):
  def __init__(self, input_length):
    super(Generator, self).__init__()
    self.linear_layer = nn.Linear(input_length, input_length)
    self.activation = nn.Sigmoid()
  
  def forward(self, x):
    return self.activation(self.linear_layer(x)).squeeze(1)

class Discriminator(nn.Module):
  def __init__(self, input_length):
    super(Discriminator, self).__init__()
    self.linear_layer = nn.Linear(input_length, 1)
    self.activation = nn.Sigmoid()
  
  def forward(self, x):
    return self.activation(self.linear_layer(x)).squeeze(1)

# Training 

In [0]:

def train(max_number = 128, batch_size = 16, training_steps = 1000):

  input_length = int(math.log(max_number, 2))

  generator = Generator(input_length)
  discriminator = Discriminator(input_length)

  generator_optimizer = torch.optim.Adam(generator.parameters(), lr = training_parameters["generator_lr"])
  discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr = training_parameters["discriminator_lr"])

  loss = nn.BCELoss()

  for training_step in range(training_steps):
    
    # Generator

    generator_optimizer.zero_grad()

    # Creating noise of #input_length bits
    noise = torch.randint(0, 10, size=(batch_size, input_length)).float()
    generated_data = generator(noise)

    true_labels, true_data = generate_training_data(max_number, batch_size=batch_size)
    
    discriminator_output_on_generated_data = discriminator(generated_data).view(batch_size)
    generator_loss = loss(discriminator_output_on_generated_data, true_labels)
    generator_loss.backward()
    generator_optimizer.step()

    # Discriminator
    '''
    It is important to zero.grad() the gradients for discriminator since in the above steps gradients were accumulated 
    but we don't want to update weights of discriminator in that step.

    '''
    discriminator_optimizer.zero_grad()
    
    discriminator_output_for_true_data = discriminator(true_data)
    true_discriminator_loss = loss(discriminator_output_for_true_data, true_labels)


    '''
     It is important to note that when passing in the generated data we want to detach the gradients. 
     We do this because we are not training the generator we are just focused on the discriminator.
    '''

    discriminator_output_for_generated_data = discriminator(generated_data.detach())
    generator_discriminator_loss = loss(
        discriminator_output_for_generated_data, torch.zeros(batch_size)
    )
    discriminator_loss = (
        true_discriminator_loss + generator_discriminator_loss
    ) / 2
    
    discriminator_loss.backward()
    discriminator_optimizer.step()

    if training_step % 10000 == 0:
        print("Training Steps Completed: ", training_step)

  return generator, discriminator



# Training Parameters

In [172]:
training_parameters = {
    "max_number": 128,
    "batch_size": 16,
    "generator_lr": 0.001,
    "discriminator_lr": 0.001,
}
generator, discriminator = train(training_steps = 100000)

Training Steps Completed:  0
Training Steps Completed:  10000
Training Steps Completed:  20000
Training Steps Completed:  30000
Training Steps Completed:  40000
Training Steps Completed:  50000
Training Steps Completed:  60000
Training Steps Completed:  70000
Training Steps Completed:  80000
Training Steps Completed:  90000


# Let's see what the generator outputs

In [152]:
with torch.no_grad():
  input_length = int(math.log(training_parameters["max_number"], 2))
  noise = torch.randint(0, 10, size=(2, input_length)).float()
  print(generator(noise))

tensor([[6.3260e-01, 8.3551e-03, 9.7807e-01, 9.9315e-01, 9.5327e-02, 7.8922e-01,
         1.7496e-06],
        [6.1768e-01, 6.5238e-04, 9.8244e-01, 9.9712e-01, 2.4946e-03, 9.9868e-01,
         2.9306e-09]])


# So, it generates a float number for every bit - it is basically trying to replicate the training data. 

# We take a threshold and assign it as 0 OR 1.

In [0]:
def convert_float_representation_to_binary_representation(float_representations, threshold = 0.5):
  binary_representations = []
  for float_representation in float_representations:
    binary_representations.append((float_representation >= threshold).int())
     
  return binary_representations

# Convert binary to decimal


In [154]:
def convert_binary_to_decimal(n):
  n = n.tolist()
  n = [str(i) for i in n]
  n = "".join(n)
  return int(n,2)
convert_binary_to_decimal(torch.tensor([0,0,1,0])) 

2

In [174]:
with torch.no_grad():
  input_length = int(math.log(training_parameters["max_number"], 2))
  noise = torch.randint(0, 10, size=( training_parameters["batch_size"], input_length)).float()
  binary_reps = convert_float_representation_to_binary_representation(generator(noise))
  for i in range(len(binary_reps)):
    binary_rep = binary_reps[i]
    print("when noise is: ",(noise[i]))
    # print(binary_rep)
    print("Generated number: ", convert_binary_to_decimal(binary_rep))
    print("---")



when noise is:  tensor([4., 9., 7., 6., 3., 4., 5.])
Generated number:  84
---
when noise is:  tensor([1., 4., 7., 6., 4., 3., 2.])
Generated number:  84
---
when noise is:  tensor([4., 9., 6., 7., 7., 8., 9.])
Generated number:  84
---
when noise is:  tensor([8., 7., 4., 2., 5., 4., 8.])
Generated number:  84
---
when noise is:  tensor([6., 0., 3., 8., 4., 9., 6.])
Generated number:  84
---
when noise is:  tensor([0., 4., 9., 4., 2., 5., 9.])
Generated number:  84
---
when noise is:  tensor([1., 6., 2., 3., 9., 6., 9.])
Generated number:  84
---
when noise is:  tensor([5., 9., 6., 7., 4., 1., 6.])
Generated number:  92
---
when noise is:  tensor([5., 0., 2., 1., 4., 3., 9.])
Generated number:  84
---
when noise is:  tensor([5., 3., 4., 3., 1., 3., 7.])
Generated number:  84
---
when noise is:  tensor([8., 5., 7., 7., 5., 5., 5.])
Generated number:  84
---
when noise is:  tensor([8., 2., 9., 6., 3., 3., 2.])
Generated number:  84
---
when noise is:  tensor([3., 8., 8., 1., 7., 3., 4.])

---
---
# Let's try with a single digit noise but in that case there are only 10 possibilities of noise and hence in the output





In [176]:
import torch.nn as nn

class Generator(nn.Module):
  def __init__(self, input_length):
    super(Generator, self).__init__()
    self.linear_layer = nn.Linear(1, input_length)
    self.activation = nn.Sigmoid()
  
  def forward(self, x):
    return self.activation(self.linear_layer(x)).squeeze(1)

class Discriminator(nn.Module):
  def __init__(self, input_length):
    super(Discriminator, self).__init__()
    self.linear_layer = nn.Linear(input_length, 1)
    self.activation = nn.Sigmoid()
  
  def forward(self, x):
    return self.activation(self.linear_layer(x)).squeeze(1)


def train(max_number = 128, batch_size = 16, training_steps = 1000):

  input_length = int(math.log(max_number, 2))

  generator = Generator(input_length)
  discriminator = Discriminator(input_length)

  generator_optimizer = torch.optim.Adam(generator.parameters(), lr = training_parameters["generator_lr"])
  discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr = training_parameters["discriminator_lr"])

  loss = nn.BCELoss()

  for training_step in range(training_steps):
    
    # Generator

    generator_optimizer.zero_grad()

    # Creating noise of #input_length bits
    noise = torch.randint(0, 10, size=(batch_size, 1)).float()
    generated_data = generator(noise)

    true_labels, true_data = generate_training_data(max_number, batch_size=batch_size)
    
    discriminator_output_on_generated_data = discriminator(generated_data).view(batch_size)
    generator_loss = loss(discriminator_output_on_generated_data, true_labels)
    generator_loss.backward()
    generator_optimizer.step()

    # Discriminator
    '''
    It is important to zero.grad() the gradients for discriminator since in the above steps gradients were accumulated 
    but we don't want to update weights of discriminator in that step.

    '''
    discriminator_optimizer.zero_grad()
    
    discriminator_output_for_true_data = discriminator(true_data)
    true_discriminator_loss = loss(discriminator_output_for_true_data, true_labels)


    '''
     It is important to note that when passing in the generated data we want to detach the gradients. 
     We do this because we are not training the generator we are just focused on the discriminator.
    '''

    discriminator_output_for_generated_data = discriminator(generated_data.detach())
    generator_discriminator_loss = loss(
        discriminator_output_for_generated_data, torch.zeros(batch_size)
    )
    discriminator_loss = (
        true_discriminator_loss + generator_discriminator_loss
    ) / 2
    
    discriminator_loss.backward()
    discriminator_optimizer.step()

    if training_step % 1000 == 0:
        print("Training Steps Completed: ",training_step)

  return generator, discriminator

training_parameters = {
    "max_number": 128,
    "batch_size": 16,
    "generator_lr": 0.001,
    "discriminator_lr": 0.001,
}
generator, discriminator = train(training_steps = 10000)



Training Steps Completed:  0
Training Steps Completed:  1000
Training Steps Completed:  2000
Training Steps Completed:  3000
Training Steps Completed:  4000
Training Steps Completed:  5000
Training Steps Completed:  6000
Training Steps Completed:  7000
Training Steps Completed:  8000
Training Steps Completed:  9000


In [177]:
with torch.no_grad():
  input_length = int(math.log(training_parameters["max_number"], 2))
  noise = torch.tensor(range(0,10)).float().unsqueeze(1)
  binary_reps = convert_float_representation_to_binary_representation(generator(noise))
  for i in range(len(binary_reps)):
    binary_rep = binary_reps[i]
    print("when noise is: ",int(noise[i][0].item()))
    # print(binary_rep)
    print("Generated number: ", convert_binary_to_decimal(binary_rep))
    print("---")


when noise is:  0
Generated number:  48
---
when noise is:  1
Generated number:  48
---
when noise is:  2
Generated number:  48
---
when noise is:  3
Generated number:  112
---
when noise is:  4
Generated number:  112
---
when noise is:  5
Generated number:  112
---
when noise is:  6
Generated number:  112
---
when noise is:  7
Generated number:  112
---
when noise is:  8
Generated number:  112
---
when noise is:  9
Generated number:  112
---


# Noise is a floating point number in the range (0,1) for every bit

In [0]:
import torch.nn as nn

class Generator(nn.Module):
  def __init__(self, input_length):
    super(Generator, self).__init__()
    self.linear_layer = nn.Linear(input_length, input_length)
    self.activation = nn.Sigmoid()
  
  def forward(self, x):
    return self.activation(self.linear_layer(x)).squeeze(1)

class Discriminator(nn.Module):
  def __init__(self, input_length):
    super(Discriminator, self).__init__()
    self.linear_layer = nn.Linear(input_length, 1)
    self.activation = nn.Sigmoid()
  
  def forward(self, x):
    return self.activation(self.linear_layer(x)).squeeze(1)


def train(max_number = 128, batch_size = 16, training_steps = 1000):

  input_length = int(math.log(max_number, 2))

  generator = Generator(input_length)
  discriminator = Discriminator(input_length)

  generator_optimizer = torch.optim.Adam(generator.parameters(), lr = training_parameters["generator_lr"])
  discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr = training_parameters["discriminator_lr"])

  loss = nn.BCELoss()

  for training_step in range(training_steps):
    
    # Generator

    generator_optimizer.zero_grad()

    # Creating noise of #input_length bits
    # noise = torch.randint(0, 10, size=(batch_size, input_length)).float()
    noise = torch.rand(batch_size, input_length)
    generated_data = generator(noise)

    true_labels, true_data = generate_training_data(max_number, batch_size=batch_size)
    
    discriminator_output_on_generated_data = discriminator(generated_data).view(batch_size)
    generator_loss = loss(discriminator_output_on_generated_data, true_labels)
    generator_loss.backward()
    generator_optimizer.step()

    # Discriminator
    '''
    It is important to zero.grad() the gradients for discriminator since in the above steps gradients were accumulated 
    but we don't want to update weights of discriminator in that step.

    '''
    discriminator_optimizer.zero_grad()
    
    discriminator_output_for_true_data = discriminator(true_data)
    true_discriminator_loss = loss(discriminator_output_for_true_data, true_labels)


    '''
     It is important to note that when passing in the generated data we want to detach the gradients. 
     We do this because we are not training the generator we are just focused on the discriminator.
    '''

    discriminator_output_for_generated_data = discriminator(generated_data.detach())
    generator_discriminator_loss = loss(
        discriminator_output_for_generated_data, torch.zeros(batch_size)
    )
    discriminator_loss = (
        true_discriminator_loss + generator_discriminator_loss
    ) / 2
    
    discriminator_loss.backward()
    discriminator_optimizer.step()

    if training_step % 10000 == 0:
        print("Training Steps Completed: ", training_step)

  return generator, discriminator


training_parameters = {
    "max_number": 128,
    "batch_size": 16,
    "generator_lr": 0.001,
    "discriminator_lr": 0.001,
}
generator, discriminator = train(training_steps = 100000)


In [181]:
with torch.no_grad():
  input_length = int(math.log(training_parameters["max_number"], 2))
  # noise = torch.randint(0, 10, size=( training_parameters["batch_size"], input_length)).float()
  noise = torch.rand(training_parameters["batch_size"], input_length)

  binary_reps = convert_float_representation_to_binary_representation(generator(noise))
  for i in range(len(binary_reps)):
    binary_rep = binary_reps[i]
    print("when noise is: ",(noise[i]))
    # print(binary_rep)
    print("Generated number: ", convert_binary_to_decimal(binary_rep))
    print("---")



when noise is:  tensor([0.8106, 0.4741, 0.9136, 0.5192, 0.9628, 0.9755, 0.3203])
Generated number:  70
---
when noise is:  tensor([0.4000, 0.0698, 0.3873, 0.1064, 0.1652, 0.2391, 0.0561])
Generated number:  70
---
when noise is:  tensor([0.8604, 0.2216, 0.6364, 0.6040, 0.9226, 0.5308, 0.1179])
Generated number:  86
---
when noise is:  tensor([0.7048, 0.5569, 0.5531, 0.2525, 0.4057, 0.9604, 0.0672])
Generated number:  70
---
when noise is:  tensor([0.5892, 0.3262, 0.2325, 0.6755, 0.6120, 0.8597, 0.7805])
Generated number:  84
---
when noise is:  tensor([0.6532, 0.9843, 0.7997, 0.1316, 0.8558, 0.8229, 0.4558])
Generated number:  70
---
when noise is:  tensor([0.3727, 0.7715, 0.9897, 0.5178, 0.2221, 0.4936, 0.5569])
Generated number:  70
---
when noise is:  tensor([0.6527, 0.7246, 0.5031, 0.6169, 0.8425, 0.8674, 0.5066])
Generated number:  86
---
when noise is:  tensor([0.3972, 0.7237, 0.0221, 0.8476, 0.5545, 0.9366, 0.7438])
Generated number:  86
---
when noise is:  tensor([0.7147, 0.758